In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
quiz_df = pd.read_csv('/content/drive/My Drive/SIADS 699: Capstone/k8_quizzes - 2-10-25.csv')
prob_df = pd.read_csv('/content/drive/My Drive/SIADS 699: Capstone/k8_probs - 2-10-25.csv')

In [ ]:
# quiz data cleaning

# convert timestamp column to pd datetime, extract date and time in separate columns
def clean_time_cols(df, time_col):
  df['timestamp'] = pd.to_datetime(df[time_col])
  df['date'] = df['timestamp'].dt.date
  df['time'] = df['timestamp'].dt.time
  return df

quiz_df = clean_time_cols(quiz_df, 'quiz_time')

# drop invalid usernames
invalid_users = ['pmb1', 'gr1', 'gr2', 'gr3', 'gr5', 'gr4', 'gr7', 'gr6', 'gr8', 'jaydenb', 'Zoe', 'pmb112358', 'pmb', 'Wizzieweed', 'Nefoli']
def clean_users(df, invalid_users):
  df = df[~df['username'].isin(invalid_users)]
  return df

quiz_df = clean_users(quiz_df, invalid_users)

# create percent correct column
quiz_df['percent_correct'] = quiz_df['num_correct'] / quiz_df['num_total']

# rearrange columns
quiz_df = quiz_df[['id', 'username', 'quiz_name', 'timestamp', 'date', 'time', 'num_correct', 'num_total', 'percent_correct']]

<ipython-input-3-1d6389df71e3>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quiz_df['percent_correct'] = quiz_df['num_correct'] / quiz_df['num_total']


In [ ]:
# create mastery df from txt file

# import txt file
mastery_df = pd.read_csv('/content/drive/My Drive/SIADS 699: Capstone/mastery.txt', sep=",", header=None, names=["username", "quiz_name", "quiz_category", "mastery_date", "grade"])

# convert timestamp column to pd datetime
mastery_df = clean_time_cols(mastery_df, 'mastery_date')

# drop invalid usernames
mastery_df = clean_users(mastery_df, invalid_users)

# drop unneeded columns and rearrange
mastery_df = mastery_df[['username', 'quiz_name', 'grade', 'timestamp', 'date', 'time']]

In [ ]:
# create binary 'mastery' column in quiz_df using mastery_df
# for each student, quiz_name, mastery will be 0 if quiz taken before mastery date, 1 if taken after

# merge quiz_df and mastery_df on username and quiz_name
quiz_df = quiz_df.merge(
    mastery_df[['username', 'quiz_name', 'timestamp', 'date', 'time']],
    on=['username', 'quiz_name'],
    how='left',
    suffixes=('', '_mastery')
)

# rename columns
quiz_df.rename(columns={
    'timestamp_mastery': 'mastery_timestamp',
    'date_mastery': 'mastery_date',
    'time_mastery': 'mastery_time'
}, inplace=True)

# create and populate the binary mastery column
quiz_df['mastery'] = (
    (quiz_df['mastery_timestamp'].notna()) &
    (quiz_df['timestamp'] >= quiz_df['mastery_timestamp'])
).astype(int)

In [ ]:
# problem data cleaning

# convert timestamp column to pd datetime, extract date and time in separate columns, handle quiz_time column
prob_df = clean_time_cols(prob_df, 'prob_time')
prob_df['quiz_time'] = prob_df['quiz_time'].replace('0000-00-00 00:00:00.000000', pd.NaT)
prob_df["quiz_time"] = pd.to_datetime(prob_df["quiz_time"], errors="coerce")

# drop invalid usernames
prob_df = clean_users(prob_df, invalid_users)

# convert Y/N entries in correct column to binary 1/0
prob_df['correct'] = prob_df['correct'].map({'N': 0, 'Y': 1})

# create binary practice column, 1 if timestamp is NaT and 0 if timestamp has a value
for _, row in prob_df.iterrows():
  if pd.isna(row['quiz_time']):
    prob_df.at[_, 'practice'] = int(1)
  else:
    prob_df.at[_, 'practice'] = int(0)

# rearrange columns
prob_df = prob_df[['username', 'timestamp', 'date', 'time', 'correct', 'category', 'code', 'quiz_time', 'practice']]

<ipython-input-6-68772460a035>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prob_df['correct'] = prob_df['correct'].map({'N': 0, 'Y': 1})


In [ ]:
# convert category and code columns into one using php 'dictionary' so that prob_name entries match quiz_name entries

# create quiz_probs_dict from a php array... did use chatGPT to convert to save time from typing/regex
quiz_probs_dict = {
    "COM1": ['com_gr1', 'com_gr1', 'com_gr1', 'com_gr1', 'com_gr1'],
    "COM2": ['com_gr2', 'com_gr2', 'com_gr2', 'com_gr2', 'com_gr2'],
    "COM3": ["com_w_gr3", "com_f_gr3", "com_w_gr3", "com_f_gr3"],
    "COM4": ["com_w_gr4", "com_f_gr4", "com d_gr4", "com_fd_gr4", "com_conv_fd"],
    "COM5": ["com_fp", "com_dp", "com_conv_fp", "com_conv_dp"],
    "AS": ['as_pv', 'as_rnd', 'as_add', 'as_sub'],
    "AS1": ["as_pv_2D", "as_add_2Dpus10s", "as_add_2D", "as_sub_2Dminus10s", "as_sub_2D"],
    "AS2": ["as_pv_3D", "as_add_3Dplus10s", "as_add_3D", "as_sub_3Dminus10s", "as_sub_3D"],
    "AS3": ["as_pv_4D", "as_rnd_4D", "as_add_4Dplus10s", "as_add_4D", "as_sub_4Dminus10s", "as_sub_4D"],
    "MD": ["md_2x2", "md_1x5", "md_0sx0s", "md_5by1", "md_5by0s"],
    "MD2": ["md_mult_234510", "md_div_234510", "md_mult_234510", "md_div_234510"],
    "MD3": ["md_mult_6789", "md_div_6789", "md_mult_1D2D", "md_div_3Dby1D"],
    "MD5": ["md_mult_exp", "md_2x2", "md_1x5", "md_0sx0s", "md_5by1", "md_5by0s"],
    "ME": ["me_if", "me_fi", "me_ms", "me_sm", "me_add", "me_sub", "me_mult", "me_div"],
    "AP": ["ap_ra", "ap_rp", "ap_rs", "ap_2ra", "ap_2rp", "ap_2rs"],
    "AP5": ["ap_tri", "ap_tri", "ap_rpv", "ap_rps"],
    "LG3": ["lg_bar", "lg_bar", "lg_bar", "lg_bar", "lg_bar"],
    "LG": ["lg_tg", "lg_gt", "lg_tg", "lg_patt"],
    "LG5": ["lg_lin", "lg_lin_rate", "lg_lin", "lg_lin_rate", "lg_lin", "lg_lin_rate"],
    "FR": ["fr_r", "fr_equiv", "fr_sim", "fr_im", "fr_a", "fr_s", "fr_m", "fr_d"],
    "FR3": ["fr_name", "fr_as_gr3", "fr_name", "fr_as_gr3"],
    "FR5": ["fr_equiv_gr5", "fr_as_gr5", "fr_mult_gr5", "fr_recip", "fr_div_gr5"],
    "DEC": ["dec_r", "dec_a", "dec_s", "dec_m", "dec_d"],
    "DEC5": ["dec_rnd3dp", "dec_as_thou", "dec_as_diffdp", "dec_md_thou", "dec_md_pow10", "dec_md_2sigdig", "dec_pv"],
    "PER": ["per_per", "per_per", "per_per", "per_id", "per_id", "per_id"],
    "OTH5": ["oth_pf", "oth_exp_np", "oth_exp_p", "oth_pf", "oth_exp_np", "oth_exp_p"],
    "PL5": ['as_pv', 'as_rnd', 'as_add', 'as_sub', "me_sub", "me_div", "ap_2ra", "ap_2rp"],
    "COM6": ['com_nn', 'com_nn', 'com_nn', 'com_nn', 'com_nn'],
    "AS6": ['as_add_nn', 'as_sub_nn', 'as_add_nn', 'as_sub_nn', 'as_add_nn', 'as_sub_nn'],
    "AP8": ['ap_pt', 'ap_pt', 'ap_pt', 'ap_pt'],
    "MD6": ["md_mult_nn", "md_div_nn", "md_mult_nn", "md_div_nn"],
    "DEC6": ['dec_div_1sd', 'dec_div_1sd', 'dec_div_1sd', 'dec_div_1sd'],
    "FR6": ['fr_div_f', 'fr_div_f', 'fr_div_f', 'fr_div_f'],
    "PRO6": ['pro_pro', 'pro_pro', 'pro_pro', 'pro_pro'],
    "FOR6": ['for_a_par', 'for_a_trap', 'for_v_cube', 'for_sa_cube', 'for_v_rp', 'for_sa_rp'],
    "FOR7": ['for_speed', 'for_c_circle', 'for_a_circle', 'for_v_pr'],
    "FOR8": ['for_v_pyr', 'for_v_cyl', 'for_v_cone', 'for_sa_sph', 'for_v_sph'],
    "EXP6": ['exp_basic', 'exp_basic', 'exp_basic', 'exp_basic'],
    "EXP7": ['exp_sr_int', 'exp_sr_dec', 'exp_sr_int', 'exp_sr_dec'],
    "EXP8": ['exp_sr_irr', 'exp_simp', 'exp_pl', 'exp_ql', 'exp_nl', 'exp_zl', 'exp_pp'],
    "LE6": ['le_as6', 'e_md6', 'le_as6', 'e_md6', 'le_as6', 'e_md6'],
    "LE7": ['le_as7', 'le_md7', 'le_two7', 'le_three7', 'le_clt7'],
    "LE8": ['le_comp8', 'le_denom8', 'le_comp8', 'le_denom8', 'le_comp8', 'le_denom8'],
    "LI6": ['li_gr', 'li_is6', 'li_gr', 'li_is6'],
    "LI7": ['li_solgr', 'li_is7', 'li_solgr', 'li_is7'],
    "LF7": ['lf_AT1', 'lf_GT', 'lf_AG1', 'lf_TG5', 'lf_TS5', 'lf_GS', 'lf_TS2', 'lf_AS1', 'lf_ST', 'lf_SG'],
    "LF8": ['lf_ES', 'lf_ET', 'lf_EG', 'lf_AE1', 'lf_SE', 'lf_GE', 'lf_TE5', 'lf_TE2', 'lf_TG2', 'lf_TT'],
    "OMS6": ['oms_absval', 'oms_ratio', 'oms_expr', 'oms_expr'],
    "OMS7": ['oms_fact', 'oms_gcf', 'oms_sqfact', 'oms_mults', 'oms_lcm'],
    "GMS6": ['gms_tg', 'gms_gt', 'gms_dist1', 'gms_gt', 'gms_tg', 'gms_hist', 'gms_hist'],
    "GMS7": ['gms_nl', 'gms_nl_mult', 'gms_nl_mult', 'gms_nl_dec', 'gms_nl_fr', 'gms_nl_per'],
    "GMS8": ['gms_dist2', 'gms_scat', 'gms_dist2', 'gms_scat', 'gms_dist2', 'gms_scat'],
    "MF2": ['mf_as'],
    "MF3": ['mf_md']
}

# reverse mapping
code_to_category = {code: category for category, codes in quiz_probs_dict.items() for code in codes}

# create prob_name column
prob_df["prob_name"] = prob_df["code"].map(code_to_category)

# rearrange columns
prob_df = prob_df[['username', 'timestamp', 'date', 'time', 'correct', 'prob_name', 'quiz_time', 'practice']]

<ipython-input-7-97b31dc0bce7>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prob_df["prob_name"] = prob_df["code"].map(code_to_category)


In [ ]:
# join with mastery to get mastery column for each problem
prob_df = prob_df.merge(
    mastery_df[['username', 'quiz_name', 'timestamp', 'date', 'time']],
    left_on=['username', 'prob_name'],
    right_on=['username', 'quiz_name'],
    how='left',
    suffixes=('', '_mastery')
)

# rename columns
prob_df.rename(columns={
    'timestamp_mastery': 'mastery_timestamp',
    'date_mastery': 'mastery_date',
    'time_mastery': 'mastery_time'
}, inplace=True)

# create and populate the binary mastery column
prob_df['mastery'] = (
    (prob_df['mastery_timestamp'].notna()) &
    (prob_df['timestamp'] >= prob_df['mastery_timestamp'])
).astype(int)

# drop unneeded columns
prob_df = prob_df.drop(columns=['quiz_name'])

In [ ]:
prob_df.head(100)

,username,timestamp,date,time,correct,prob_name,quiz_time,practice,mastery_timestamp,mastery_date,mastery_time,mastery
0,CharlotteC14,2024-07-24 11:34:18,2024-07-24,11:34:18,0,COM1,NaT,1.0,2024-12-10 14:18:10,2024-12-10,14:18:10,0
1,CharlotteC14,2024-07-24 11:35:40,2024-07-24,11:35:40,0,COM1,NaT,1.0,2024-12-10 14:18:10,2024-12-10,14:18:10,0
2,CharlotteC14,2024-07-24 11:35:58,2024-07-24,11:35:58,0,COM1,NaT,1.0,2024-12-10 14:18:10,2024-12-10,14:18:10,0
3,CharlotteC14,2024-07-24 11:36:13,2024-07-24,11:36:13,0,COM1,NaT,1.0,2024-12-10 14:18:10,2024-12-10,14:18:10,0
4,CharlotteC14,2024-07-24 11:42:08,2024-07-24,11:42:08,0,COM1,NaT,1.0,2024-12-10 14:18:10,2024-12-10,14:18:10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,KnoxK2,2024-07-24 12:30:04,2024-07-24,12:30:04,0,MD5,2024-07-24 12:19:32,0.0,NaT,NaN,NaN,0
96,PaytonP4,2024-07-24 12:30:08,2024-07-24,12:30:08,1,MD2,NaT,1.0,2024-08-06 12:16:04,2024-08-06,12:16:04,0
97,PaytonP4,2024-07-24 12:30:16,2024-07-24,12:30:16,1,MD2,NaT,1.0,2024-08-06 12:16:04,2024-08-06,12:16:04,0
98,OarielO,2024-07-24 12:30:46,2024-07-24,12:30:46,0,AS3,2024-07-24 12:30:23,0.0,2024-11-19 12:39:58,2024-11-19,12:39:58,0


In [ ]:
# create a probs_before_mastery column that counts the problems each student completed for each category before achieving mastery
prob_df = prob_df.sort_values(by=['username', 'prob_name', 'timestamp'])
prob_df['probs_before_mastery'] = prob_df.groupby(['username', 'prob_name']).apply(
    lambda group: (group['timestamp'] < group['mastery_timestamp']).cumsum()
).reset_index(drop=True)
prob_df['probs_before_mastery'] = prob_df['probs_before_mastery'].fillna(0)

# create a time_before_mastery column that calculates the time from the students first attempt at a problem type to when they achieve mastery
first_attempts = prob_df.groupby(['username', 'prob_name'])['timestamp'].transform('min')
prob_df['time_to_mastery'] = prob_df['mastery_timestamp'] - first_attempts
prob_df.loc[prob_df['mastery_timestamp'].isna(), 'time_to_mastery'] = pd.NaT
prob_df['time_to_mastery'] = prob_df['time_to_mastery'].fillna(pd.Timedelta(seconds=0))

<ipython-input-10-f186ebe359c5>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  prob_df['probs_before_mastery'] = prob_df.groupby(['username', 'prob_name']).apply(


In [ ]:
# merge prob_df and quiz_df to get one central dataframe with all data
merged_df = prob_df.merge(
    quiz_df,
    left_on=['username', 'quiz_time'],
    right_on=['username', 'timestamp'],
    how='left'
)

In [ ]:
# merged_df cleaning

# rename any duplicated columns
merged_df.rename(columns={
    'timestamp_x': 'prob_timestamp',
    'date_x': 'prob_date',
    'time_x': 'prob_time',
    'quiz_time': 'inherited_quiz_time',
    'mastery_timestamp_x': 'mastery_timestamp',
    'mastery_date_x': 'mastery_date',
    'mastery_time_x': 'mastery_time',
    'mastery_x': 'mastery',
    'timestamp_y': 'quiz_timestamp',
    'date_y': 'quiz_date',
    'time_y': 'quiz_time'
}, inplace=True)

# drop redundant columns
merged_df = merged_df.drop(columns=['mastery_timestamp_y', 'mastery_date_y', 'mastery_time_y', 'mastery_y', 'inherited_quiz_time', 'id', 'quiz_name'])

# fill NaN values in quiz parameters for practice problems
merged_df.loc[merged_df['num_correct'].isna(), 'num_correct'] = merged_df['correct']
merged_df.loc[merged_df['num_total'].isna(), 'num_total'] = 1
merged_df.loc[merged_df['percent_correct'].isna(), 'percent_correct'] = merged_df['correct']

In [ ]:
merged_df.head(20)

,username,prob_timestamp,prob_date,prob_time,correct,prob_name,practice,mastery_timestamp,mastery_date,mastery_time,mastery,probs_before_mastery,time_to_mastery,quiz_timestamp,quiz_date,quiz_time,num_correct,num_total,percent_correct
0,AyaA2,2024-10-09 09:31:36,2024-10-09,09:31:36,0,AP,0.0,NaT,NaN,NaN,0,0.0,0 days,NaT,NaN,NaN,0.0,1.0,0.0
1,AyaA2,2024-10-09 09:35:21,2024-10-09,09:35:21,1,AP,0.0,NaT,NaN,NaN,0,0.0,0 days,NaT,NaN,NaN,1.0,1.0,1.0
2,AyaA2,2024-10-09 09:36:39,2024-10-09,09:36:39,1,AP,0.0,NaT,NaN,NaN,0,0.0,0 days,2024-10-09 09:35:31,2024-10-09,09:35:31,3.0,6.0,0.5
3,AyaA2,2024-10-09 09:39:09,2024-10-09,09:39:09,1,AP,0.0,NaT,NaN,NaN,0,0.0,0 days,2024-10-09 09:35:31,2024-10-09,09:35:31,3.0,6.0,0.5
4,AyaA2,2024-10-09 09:40:47,2024-10-09,09:40:47,1,AP,0.0,NaT,NaN,NaN,0,0.0,0 days,2024-10-09 09:35:31,2024-10-09,09:35:31,3.0,6.0,0.5
5,AyaA2,2024-10-09 09:46:59,2024-10-09,09:46:59,0,AP,0.0,NaT,NaN,NaN,0,0.0,0 days,2024-10-09 09:35:31,2024-10-09,09:35:31,3.0,6.0,0.5
6,AyaA2,2024-10-15 10:23:50,2024-10-15,10:23:50,1,AP,0.0,NaT,NaN,NaN,0,0.0,0 days,NaT,NaN,NaN,1.0,1.0,1.0
7,AyaA2,2024-10-15 10:26:15,2024-10-15,10:26:15,1,AP,0.0,NaT,NaN,NaN,0,0.0,0 days,NaT,NaN,NaN,1.0,1.0,1.0
8,AyaA2,2024-10-15 10:26:44,2024-10-15,10:26:44,0,AP,0.0,NaT,NaN,NaN,0,0.0,0 days,NaT,NaN,NaN,0.0,1.0,0.0
9,AyaA2,2024-11-18 09:57:55,2024-11-18,09:57:55,1,AP,1.0,NaT,NaN,NaN,0,0.0,0 days,NaT,NaN,NaN,1.0,1.0,1.0


In [ ]:
# convert category and code columns into one using php 'dictionary' so that prob_name entries match quiz_name entries

# create quiz_probs_dict from a php array... did use chatGPT to convert to save time from typing/regex
prob_grades_dict = {
    "COM1": 1,
    "COM2": 2,
    "COM3": 3,
    "COM4": 4,
    "COM5": 5,
    "AS": 4,
    "AS1": 1,
    "AS2": 2,
    "AS3": 3,
    "MD": 4,
    "MD2": 2,
    "MD3": 3,
    "MD5": 5,
    "ME": 4,
    "AP": 4,
    "AP5": 5,
    "LG3": 3,
    "LG": 4,
    "LG5": 5,
    "FR": 4,
    "FR3": 3,
    "FR5": 5,
    "DEC": 4,
    "DEC5": 5,
    "PER": 5,
    "OTH5": 5,
    "PL5": 5,
    "COM6": 6,
    "AS6": 6,
    "AP8": 8,
    "MD6": 6,
    "DEC6": 6,
    "FR6": 6,
    "PRO6": 6,
    "FOR6": 6,
    "FOR7": 7,
    "FOR8": 8,
    "EXP6": 6,
    "EXP7": 7,
    "EXP8": 8,
    "LE6": 6,
    "LE7": 7,
    "LE8": 8,
    "LI6": 6,
    "LI7": 7,
    "LF7": 7,
    "LF8": 8,
    "OMS6": 6,
    "OMS7": 7,
    "GMS6": 6,
    "GMS7": 7,
    "GMS8": 8,
    "MF2": 2,
    "MF3": 3
}

# create prob_name column
merged_df["prob_grade_level"] = merged_df["prob_name"].map(prob_grades_dict)

In [ ]:
# download mastery_df, quiz_df, prob_df, and merged_df as csv files and save to drive folder

merged_df.to_csv("merged_data.csv", index=False)
mastery_df.to_csv("mastery_data.csv", index=False)
quiz_df.to_csv("quiz_data.csv", index=False)
prob_df.to_csv("prob_data.csv", index=False)

from google.colab import files
files.download("merged_data.csv")
files.download("mastery_data.csv")
files.download("quiz_data.csv")
files.download("prob_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>